In [ ]:
# Take the user's parameters and put them into a Python
# dictionary structured like an Elasticsearch query:
query_body = {"query": {"bool": {"must": {"match": {"some_field": user_request } } } }}

# call the client's search() method, and have it return results
result = es.search(index="some_index", body=query_body)

# see how many "hits" it returned using the len() function
print ("total hits:", len(result["hits"]["hits"]))


A.Loading elasticsearch

In [1]:
from elasticsearch import Elasticsearch

# Adds the HTTP header 'Authorization: Basic <base64 username:password>'
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "lxDO2mQDVQPrxE3FRWAy")
)

print(es.ping())

True


In [5]:
es.indices.get_alias()

<ipython-input-5-2b814a79a8f7>:1: ElasticsearchWarning: this request accesses system indices: [.transform-internal-007, .kibana_8.4.3_001, .apm-agent-configuration, .kibana_security_session_1, .tasks, .security-7, .apm-custom-link, .kibana_task_manager_8.4.3_001, .security-profile-8, .async-search], but in a future major version, direct access to system indices will be prevented by default
  es.indices.get_alias()


ObjectApiResponse({'.kibana-event-log-8.4.3-000001': {'aliases': {'.kibana-event-log-8.4.3': {'is_write_index': True, 'is_hidden': True}}}, 'ecommerce_data': {'aliases': {}}, 'a': {'aliases': {}}, 'comments': {'aliases': {}}, 'news_headlines': {'aliases': {}}, '.transform-internal-007': {'aliases': {}}, '.kibana_8.4.3_001': {'aliases': {'.kibana': {'is_hidden': True}, '.kibana_8.4.3': {'is_hidden': True}}}, '.apm-agent-configuration': {'aliases': {}}, 'questions2': {'aliases': {}}, 'questions': {'aliases': {}}, '.kibana_security_session_1': {'aliases': {'.kibana_security_session': {'is_hidden': True}}}, '.tasks': {'aliases': {}}, 'metrics-endpoint.metadata_current_default': {'aliases': {}}, '.security-7': {'aliases': {'.security': {'is_hidden': True}}}, 'favorite_candy': {'aliases': {}}, '.apm-custom-link': {'aliases': {}}, '.kibana_task_manager_8.4.3_001': {'aliases': {'.kibana_task_manager': {'is_hidden': True}, '.kibana_task_manager_8.4.3': {'is_hidden': True}}}, '.security-profile-

In [17]:
es.indices.delete(index="questions")

ObjectApiResponse({'acknowledged': True})

B.Importing data for assignments(1,2,3)

In [7]:
import pandas as pd
df_q = pd.read_csv('C:\\Users\\A\\Downloads\\questions.csv').fillna("")

In [8]:
df_q.head(2)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,115145,967227,2009-05-31 16:20:08,CommonsWare,2022-10-29 23:14:28,https://commonsware.com,"Pennsylvania, United States","<p>Mark Murphy, founder of CommonsWare, writes...",269231,13193,...,2022-08-19 16:46:16,2022-08-19 16:46:16,"How can I fix ""unexpected element <queries> fo...",<android><android-gradle-plugin><android-manif...,7,1,30,,,CC BY-SA 4.0
1,85371,358430,2009-03-31 21:38:26,sehe,2022-10-29 15:03:44,,Netherlands,<p>I'm a developer at heart.</p>\n<p>I love C+...,34058,16475,...,2021-03-06 02:51:07,2021-03-06 02:51:07,C++20 behaviour breaking existing code with eq...,<c++><c++17><c++20><spaceship-operator>,1,4,21,,,CC BY-SA 4.0


In [9]:
df_q.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'EmailHash', 'AccountId', 'Id.1', 'PostTypeId',
       'AcceptedAnswerId', 'ParentId', 'CreationDate.1', 'DeletionDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate', 'ContentLicense'],
      dtype='object')

In [10]:
# df_q['Id']=users.id
len(df_q['Id'].unique())

51

In [11]:
# df_q['Id.1'] = posts.id
len(df_q['Id.1'].unique())

100

C.Indexing data based on the assignment

In [18]:
mapping = {
     "properties": {
            "ID": {"type": "integer"},
            "Body": {"type": "text"},
            "Tags": {"type": "keyword"}
            #"CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            #"UpVotes": {"type": "integer"},
            #"Score": {"type": "integer"},
            #"OwnerUserId": {"type": "integer"},
         }}

es.indices.create(index="questions",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions'})

In [19]:
for i,row in df_q.iterrows():
  doc = {
      "ID" : row['Id.1'], # id.1=posts.id
      "Body" : row["Body"],
      "Tags" : (row["Tags"] if row["Tags"]!= 'non' else '')
      #"CreationDate" : row["CreationDate"], no need
      #"UpVotes" : row["UpVotes"], no need
      #"Score" : row["Score"],
      #"OwnerUserId" : row["OwnerUserId"],
  }
  
  es.index(index="questions",id=i,document=doc)

C_2.Checking indexed data

In [20]:
es.get(index="questions",id=0)

ObjectApiResponse({'_index': 'questions', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'ID': 62969917, 'Body': '<p>All of a sudden, I am getting this build error in my Android project:</p>\n<pre><code>unexpected element &lt;queries&gt; found in &lt;manifest&gt;\n</code></pre>\n<p>How do I fix it?</p>\n', 'Tags': '<android><android-gradle-plugin><android-manifest><manifest>'}})

In [21]:
es.indices.get(index='questions')

ObjectApiResponse({'questions': {'aliases': {}, 'mappings': {'properties': {'Body': {'type': 'text'}, 'ID': {'type': 'integer'}, 'Tags': {'type': 'keyword'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'questions', 'creation_date': '1667153386414', 'number_of_replicas': '1', 'uuid': 'fRdaMkObQta1zhDPbODuTA', 'version': {'created': '8040399'}}}}})

In [22]:
es.indices.refresh(index="questions")


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [23]:
es.cat.count(index="questions", format="json")

ListApiResponse([{'epoch': '1667153446', 'timestamp': '18:10:46', 'count': '100'}])

In [30]:
res = es.get(index="questions", id=1)
res

ObjectApiResponse({'_index': 'questions', '_id': '1', '_version': 1, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'ID': 65648897, 'Body': '<p>I ran into this while debugging <a href="https://stackoverflow.com/questions/65638680/using-boost-strong-typedef-to-differentiate-arg-types-but-causing-seg-fault/65643014?noredirect=1#comment116069484_65643014">this question</a>.</p>\n<p>I trimmed it down all the way to just using <a href="https://www.boost.org/doc/libs/1_75_0/libs/utility/operators.htm" rel="noreferrer">Boost Operators</a>:</p>\n<ol>\n<li><p><strong>Compiler Explorer <a href="https://godbolt.org/z/q89brM" rel="noreferrer">C++17</a> <a href="https://godbolt.org/z/szYx9E" rel="noreferrer">C++20</a></strong></p>\n<pre><code>#include &lt;boost/operators.hpp&gt;\n\nstruct F : boost::totally_ordered1&lt;F, boost::totally_ordered2&lt;F, int&gt;&gt; {\n    /*implicit*/ F(int t_) : t(t_) {}\n    bool operator==(F const&amp; o) const { return t == o.t; }\n    bool operator

In [29]:
res['_source']['Tags']

'<c++><c++17><c++20><spaceship-operator>'

In [27]:
res.keys()

dict_keys(['_index', '_id', '_version', '_seq_no', '_primary_term', 'found', '_source'])

In [34]:
resp = es.search(index="questions", query={"match":{"Body":"program"}})

In [35]:
# first result
resp['hits']['hits'][0]


{'_index': 'questions',
 '_id': '82',
 '_score': 3.8813283,
 '_source': {'ID': 61278180,
  'Body': '<p>Consider the following demonstrative program.</p>\n\n<pre><code>#include &lt;iostream&gt;\n\nint main()\n{\n    typedef float T;\n\n    0.f.T::~T();\n}\n</code></pre>\n\n<p>This program is compiled by <code>Microsoft Visual Studio Community 2019</code>.</p>\n\n<p>But <code>clang</code> and <code>gcc</code> issue an error like this</p>\n\n<pre><code>prog.cc:7:5: error: unable to find numeric literal operator \'operator""f.T\'\n    7 |     0.f.T::~T();\n      |     ^~~~~\n</code></pre>\n\n<p>If to write the expression like <code>( 0.f ).T::~T()</code> then all three compilers compile the program.</p>\n\n<p>So a question arises: is this record <code>0.f.T::~T()</code> syntactically valid? And if not, then what syntactical rule is broken? </p>\n',
  'Tags': '<c++><syntax><floating-point><c++17><pseudo-destructor>'}}

In [36]:
resp['hits']['hits'][0]['_source']

{'ID': 61278180,
 'Body': '<p>Consider the following demonstrative program.</p>\n\n<pre><code>#include &lt;iostream&gt;\n\nint main()\n{\n    typedef float T;\n\n    0.f.T::~T();\n}\n</code></pre>\n\n<p>This program is compiled by <code>Microsoft Visual Studio Community 2019</code>.</p>\n\n<p>But <code>clang</code> and <code>gcc</code> issue an error like this</p>\n\n<pre><code>prog.cc:7:5: error: unable to find numeric literal operator \'operator""f.T\'\n    7 |     0.f.T::~T();\n      |     ^~~~~\n</code></pre>\n\n<p>If to write the expression like <code>( 0.f ).T::~T()</code> then all three compilers compile the program.</p>\n\n<p>So a question arises: is this record <code>0.f.T::~T()</code> syntactically valid? And if not, then what syntactical rule is broken? </p>\n',
 'Tags': '<c++><syntax><floating-point><c++17><pseudo-destructor>'}

In [40]:
resp['hits']['hits'][0]['_source']['Body']

'<p>Consider the following demonstrative program.</p>\n\n<pre><code>#include &lt;iostream&gt;\n\nint main()\n{\n    typedef float T;\n\n    0.f.T::~T();\n}\n</code></pre>\n\n<p>This program is compiled by <code>Microsoft Visual Studio Community 2019</code>.</p>\n\n<p>But <code>clang</code> and <code>gcc</code> issue an error like this</p>\n\n<pre><code>prog.cc:7:5: error: unable to find numeric literal operator \'operator""f.T\'\n    7 |     0.f.T::~T();\n      |     ^~~~~\n</code></pre>\n\n<p>If to write the expression like <code>( 0.f ).T::~T()</code> then all three compilers compile the program.</p>\n\n<p>So a question arises: is this record <code>0.f.T::~T()</code> syntactically valid? And if not, then what syntactical rule is broken? </p>\n'

In [39]:
print(f"Got {resp['hits']['total']['value']} Hits:" )
for hit in resp['hits']['hits']:
    print(hit['_source']['ID'])

Got 12 Hits:
61278180
68505026
60627249
59787879
67619477
59603086
65080685
60114639
65648897
63705739


Assignment(1)

In [41]:
def search_text(text):
    results = es.search(index='questions',query={"match":{"Body":text}})
    all_hits = results['hits']['hits']

    # see how many "hits" it returned using the len() function
    print ("total hits :", results['hits']['total']['value'], "\n")
    
    # iterate the nested dictionaries inside the ["hits"]["hits"] list and print results
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "\n", doc['_source']['Body'], "\n")

In [46]:
search_text('concatenation')

total hits : 1 

DOC ID: 76 
 <p>I have existing code such as this:</p>

<pre><code>String sql = "CREATE TABLE " + tableName + " (\n" +
        "  id_ UUID DEFAULT random_uuid() PRIMARY KEY ,\n" +
        "  when_ TIMESTAMP WITHOUT TIME ZONE NOT NULL\n" +
        "  duration_ STRING NOT NULL\n" +
        ");";
</code></pre>

<p>…in a project in IntelliJ 2020.1.1 (preview) Ultimate edition where the JDK settings are configured for language level <code>14 (Preview) - Records, patters, text blocks</code>. </p>

<p>So I expected IntelliJ to offer a conversion to text blocks among the items display when I click on the yellow light bulb that appears next to this line of code. But I see no such offer.</p>

<p><a href="https://i.stack.imgur.com/f18mc.png" rel="noreferrer"><img src="https://i.stack.imgur.com/f18mc.png" alt="screen shot of pop-up menu displayed by clicking on yellow light bulb next to this line of code"></a></p>

<p>I am surprised to not find such an offer, as the JetBrains comp

Assignment(2)

In [ ]:
results = es.search(index='questions',query={"match":{"ID":63124766}})
all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Body'], "\n")

In [129]:
df['Tags'].unique()

array(['', '<android><android-gradle-plugin><android-manifest><manifest>',
       '<c++><c++17><c++20><spaceship-operator>'], dtype=object)

In [147]:
results = es.search(index='questions',query={"bool": {"must": [{"wildcard":{"Tags":"*android*" }}]}})


all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Tags'], "\n")

1
DOC ID: 6 ---> <android><android-gradle-plugin><android-manifest><manifest> 



In [148]:
results = es.search(index='questions',query={"bool": {"must_not": [{"match":{"Tags":"" }}]}})


all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Tags'], "\n")

2
DOC ID: 6 ---> <android><android-gradle-plugin><android-manifest><manifest> 

DOC ID: 23 ---> <c++><c++17><c++20><spaceship-operator> 



In [2]:
import pandas as pd
df2 = pd.read_csv('C:\\Users\\A\\Desktop\\comments.csv').fillna("")

In [3]:
df2.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserDisplayName',
       'UserId', 'ContentLicense'],
      dtype='object')

In [ ]:
results = es.search(index='questions',query={"bool": { "should": [ {"term": {"Tags": "android" }},  
      ]
    }
  })


all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->", doc['_source']['Tags'], "\n")

In [ ]:
df2['CreationDate'].unique()

In [30]:
df2['PostId'].unique()

array([59734482, 59837471, 59871821, 59928620, 60071551, 60128446,
       60020384, 60466572, 60461828, 60551320, 60602455, 60626081,
       60969045, 61009026, 61337953, 61645042, 61622333, 61736789,
       62140681, 62057615, 62431424, 62601621, 62714182, 62347506,
       62625899, 62807914, 63124766, 62734983, 64083083, 60013101,
       63580047, 63724548, 63826218, 64252780, 60405256, 64476015,
       62656145, 59623717, 63727364, 63117292, 63826219, 60202636,
       60979785, 62347613, 60266696, 64545863, 59857409, 65174592,
       62969918, 65648897, 67145373, 67414097, 61981767, 67979333,
       68152911, 68938785, 64839917, 69843565, 70011062, 70258852,
       70340785, 61147681, 61732176, 61922965, 66348974, 66993654,
       59658315, 62761032, 61531318, 68221126, 64795010, 64081011,
       63102614, 61236803, 59675191, 69271299, 70099102, 66309040,
       65379845, 69503445, 62978895, 63778277, 69875984, 64747517,
       62784897, 66405205, 64134704, 70231955, 64310139, 62969

In [8]:
len(df2['PostId'].unique())

90

In [25]:
mapping = {
     "properties": {
            "PostId": {"type": "integer"},
            "CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            "Score": {"type": "integer"},
            "Text": {"type": "text"},
         }}

es.indices.create(index="comments",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'comments'})

In [29]:
for i,row in df2.iterrows():
  doc = {
      "PostId" : row['PostId'],
      "CreationDate" : row["CreationDate"],
      "Score" : row["Score"],
      "Text" : row["Text"],
  }
  es.index(index="comments",id=i,document=doc)

In [40]:
results = es.search(index='comments',query={"bool": {"must": [{"term": {"PostId": 59837471}}],"filter":{"range":{"Score": {"gte": 5}}}}})
all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->","Score = ",doc['_source']['Score'],"\n" ,doc['_source']['Text'], "\n")

4
DOC ID: 1 ---> Score =  46 
 The RTL override is _not_ used for writing right-to-left languages. Those automatically render RTL.  The bigger problem is shared characters like numbers.  For those, the renderer has to _guess_ LTR vs RTL based on context.  Developers insert the RTL override character to force the renderer to render neutral direction characters in the desired direction. 

DOC ID: 22 ---> Score =  11 
 @Jordan.J.D yes, they should change selection appropriately in most software that handles RTL text properly. 

DOC ID: 273 ---> Score =  14 
 Do these characters change the way my browser chooses to highlight on double click as well? when I try to double click OP's string it highlights the beginning quote, and end dot and end quote, nothing in between. Interesting. 

DOC ID: 275 ---> Score =  6 
 @Moo It’s used *in the context of writing RTL languages in certain situations.* I didn’t want to go into too much detail as it’s irrelevant to this question. 



In [45]:
results = es.search(index='comments',query={"match_phrase":{"Text":"such behavior"}})
all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->",doc['_source']['Text'], "\n")

1
DOC ID: 483 ---> ...but allowing such behavior means that any programmer who can't be certain a loop would terminate, but who would be willing to have a program hang until externally killed if it fails to do so, would need to add directives to block all of the useful optimizations that would be facilitated by a parallel execution model.  Do you see any upside to the clang/LLVM view of infinite loops? 



In [53]:
results = es.search(index='comments',query={"bool": {"must": [{"match_phrase": {"Text":"programmer"}}],"filter":{"range":{"CreationDate": {"gte": "2020-09-27 10:37:31","lte": "2022-09-27 10:37:31"}}}}})
all_hits = results['hits']['hits']
print(results['hits']['total']['value'])
if results['hits']['total']['value'] == 0:
    print('NOT FOUND')  
else:
    for num, doc in enumerate(all_hits):
        print ("DOC ID:", doc["_id"], "--->","CreationDate = ",doc['_source']['CreationDate'],"\n" ,doc['_source']['Text'], "\n")

3
DOC ID: 483 ---> CreationDate =  2022-02-01 22:35:26 
 ...but allowing such behavior means that any programmer who can't be certain a loop would terminate, but who would be willing to have a program hang until externally killed if it fails to do so, would need to add directives to block all of the useful optimizations that would be facilitated by a parallel execution model.  Do you see any upside to the clang/LLVM view of infinite loops? 

DOC ID: 467 ---> CreationDate =  2022-02-04 16:41:07 
 @EricTowers: What's ironic and sad is that in the common cases where optimizations allowed by the Standard should have value,--those where either hanging or skipping the loop would be equally acceptable behaviors, but unbounded UB would not be--the only way to guard against UB is for the programmer to write the program in such a way that the loop could not be skipped, thus negating any benefit the optimization could have offered. 

DOC ID: 232 ---> CreationDate =  2022-04-06 15:09:04 
 @BenW  I